In [1]:
from utils.io import gen_id_dated, gdf_viewer, gdf_geom, gdf_merger, na_col_drop, na_line_drop, data_validation
import pandas as pd
import numpy as np
from utils.utils import update_dict

### Create data

In [2]:
t1 = pd.DataFrame({'ID':['id1','id2','id3','id4','id5','id6','id7','id8', 'id9'],'Name':['A','B','C','D','E','F','G','H','I'],
                   'Value':[1,2,2,4,np.nan,5,np.nan,8,9], 
                   'Date': ['2021-06-01','2021-06-02','2021-06-04','2021-06-07','2021-06-08','2021-06-10',
                            '2021-06-11','2021-06-04','2021-06-02'],
                   'X': [1.,4.,50.,8.,7.,11.,8.,np.nan,10], 
                   'Y': [7.,4.,2.,16.,14.,20.,11.,np.nan,6]})

In [3]:
t2 = pd.DataFrame({'ID':['id1','id2','id3','id4','id5','id6','id7','id8', 'id9'],'Name':['A','B','D','C',np.nan,'F','G','h','k'],
                   'Value':[1,3,2,2,np.nan,np.nan,6,7,9],
                   'Date': ['2021-06-01','2021-06-03','2021-06-04','2021-06-07','2021-06-09','2021-06-11',
                            '2021-06-03','2021-06-04',np.nan],
                   'X': [np.nan,4.,12.,8.,7.,11.,8.,13.,10], 
                   'Y': [np.nan,4.,20.,18.,104.,9.,1.,5.,6]})

In [4]:
t1.merge(t2, on='ID', how='outer')

,ID,Name_x,Value_x,Date_x,X_x,Y_x,Name_y,Value_y,Date_y,X_y,Y_y
0,id1,A,1.0,2021-06-01,1.0,7.0,A,1.0,2021-06-01,NaN,NaN
1,id2,B,2.0,2021-06-02,4.0,4.0,B,3.0,2021-06-03,4.0,4.0
2,id3,C,2.0,2021-06-04,50.0,2.0,D,2.0,2021-06-04,12.0,20.0
3,id4,D,4.0,2021-06-07,8.0,16.0,C,2.0,2021-06-07,8.0,18.0
4,id5,E,NaN,2021-06-08,7.0,14.0,NaN,NaN,2021-06-09,7.0,104.0
5,id6,F,5.0,2021-06-10,11.0,20.0,F,NaN,2021-06-11,11.0,9.0
6,id7,G,NaN,2021-06-11,8.0,11.0,G,6.0,2021-06-03,8.0,1.0
7,id8,H,8.0,2021-06-04,NaN,NaN,h,7.0,2021-06-04,13.0,5.0
8,id9,I,9.0,2021-06-02,10.0,6.0,k,9.0,NaN,10.0,6.0


In [21]:
np.nanmedian(t1['Value']), np.nanmean(t1['Value'])

(4.0, 4.428571428571429)

### Merging

In [5]:
m, conflicts = gdf_merger(t1, t2, on='ID', how='outer', dist_max=1., date_col='Date')

Conflict values present. Please resolve this manually !


In [6]:
#m['index'] = m['index'].astype('int')
m

,ID,X,Y,index,Value,Name,Date
0,id1,1.0,7.0,0.0,1.0,A,2021-06-01
1,id3,50.0,2.0,2.0,2.0,#conflict,2021-06-04
2,id4,8.0,16.0,3.0,NaN,#conflict,2021-06-07
3,id8,13.0,5.0,7.0,NaN,H,2021-06-04
4,id9,10.0,6.0,8.0,9.0,#conflict,2021-06-02
5,_id2_,4.0,4.0,NaN,2.0,B,2021-06-02
6,id2,4.0,4.0,NaN,3.0,B,2021-06-03
7,_id5_,7.0,14.0,NaN,NaN,E,2021-06-08
8,id5,7.0,104.0,NaN,NaN,NaN,2021-06-09
9,_id6_,11.0,20.0,NaN,5.0,F,2021-06-10


### conflicts management

In [7]:
conflicts

,Check_col,ID,Name_x,Name_y,Value_x,Value_y
2,Name,id3,C,D,2.0,2.0
3,"Value, Name",id4,D,C,4.0,2.0
7,Value,id8,H,h,8.0,7.0
8,Name,id9,I,k,9.0,9.0


In [8]:
data_validation(overall_data=m, conflict_data=conflicts, valid_dict={'Value_x':[7], 'Value_y':[3],
                                                                    'Name_y':[2,3]})

Validation done, but conflicts remain!


### Results

In [9]:
m

,ID,X,Y,index,Value,Name,Date
0,id1,1.0,7.0,0.0,1.0,A,2021-06-01
1,id3,50.0,2.0,2.0,2.0,NaN,2021-06-04
2,id4,8.0,16.0,3.0,NaN,D,2021-06-07
3,id8,13.0,5.0,7.0,NaN,H,2021-06-04
4,id9,10.0,6.0,8.0,9.0,#conflict,2021-06-02
5,_id2_,4.0,4.0,NaN,2.0,B,2021-06-02
6,id2,4.0,4.0,NaN,3.0,B,2021-06-03
7,_id5_,7.0,14.0,NaN,NaN,E,2021-06-08
8,id5,7.0,104.0,NaN,NaN,NaN,2021-06-09
9,_id6_,11.0,20.0,NaN,5.0,F,2021-06-10


In [10]:
conflicts

,Check_col,ID,Name_x,Name_y,Value_x,Value_y
8,Name,id9,I,k,9.0,9.0
